In [1]:
import pandas as pd
from langchain.chains import LLMChain, SequentialChain


In [2]:
import prompts as p
from langchain.prompts import PromptTemplate


PRODUCTS = '../../datasets/amazon_walmart.tsv'

PRODUCT_MAPPING = '../../datasets/WalAmazonDataset_sampled.csv'

SAVED_PROMPTS_PATH = '../../evals/saved_prompts_WA.csv'

# Load data
df_products = pd.read_csv(PRODUCTS, sep='\t')
df_mapping = pd.read_csv(PRODUCT_MAPPING)


df_products.fillna('', inplace=True)
df_mapping.fillna('', inplace=True)

prompt_name = 'prompt1_en_short_temp_0_gpt4-0613'
prompt = p.prompt1_en_short

MODEL = "gpt-4-0613"

MAX_TOKENS = 15


In [3]:
# generate column with prompt_name if not exists
if 'prompt_name' not in df_mapping.columns:
    df_mapping[prompt_name] = ""

In [4]:
df_mapping.head(30)

,Unnamed: 0,id1,id2,label,prompt1_en_short_temp_0_gpt3.5-0301,match_prompt_manual_fewshot_1_eval2_temp_0_gpt3.5_0301,match_prompt_auto_fewshot_1_eval2_temp_0_gpt3.5_0301,match_prompt_auto_fewshot_2_ama_wal_eval2_temp_0_gpt3.5_0301,match_prompt_auto_fewshot_ama_wal_4examples_eval2_temp_0_gpt3.5_0301,prompt1_en_short_temp_0_gpt4-0613
0,0,7784,23390,True,True,False,False,False,False,
1,1,15123,23390,True,False,False,False,False,True,
2,2,21113,23390,True,True,True,True,False,True,
3,3,7384,22973,True,True,True,True,True,True,
4,4,11344,22973,True,False,True,False,True,True,
5,5,15336,22973,True,True,True,True,True,True,
6,6,8176,24537,True,True,True,True,True,True,
7,7,14118,24537,True,True,True,True,True,True,
8,8,20398,23769,True,True,True,False,False,True,
9,9,11027,23769,True,True,True,False,False,True,


In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI


In [6]:
from dotenv import load_dotenv
import os
load_dotenv()


True

In [7]:
from langfuse.callback import CallbackHandler
handler = CallbackHandler(os.environ.get("LF_PK"), os.environ.get("LF_SK"), os.environ.get("LF_HOST"))

In [8]:
def cut_length(string ,length = 250):
    return string[0:length]

In [9]:

# product1 = "Title: " + cut_length(str(df["title_left"][1] or "none")) + "\nDescription: " + cut_length(str(df["description_left"][1] or "none"))
# product2 = "Title: " + cut_length(str(df["title_right"][1] or "none")) + "\nDescription: " + cut_length(str(df["description_right"][1] or "none"))



# product1 = "Title: " + cut_length(str(df["title_left"][1] or "none")) + "\nDescription: " + cut_length(str(df["description_left"][1] or "none")) + "\nBrand:" + str(df["brand_left"][1] or "none") 
# product2 = "Title: " + cut_length(str(df["title_right"][1] or "none")) + "\nDescription: " + cut_length(str(df["description_right"][1] or "none")) + "\nBrand:" + str(df["brand_right"][1] or "none") 

In [10]:
saved_prompts_df = pd.read_csv(SAVED_PROMPTS_PATH)

def saveResponse(prompt, response, isMatch, index):
    # create new column if not exists 
    if "prompt_" + prompt_name not in saved_prompts_df.columns:
        saved_prompts_df["prompt_" + prompt_name] = ""
        saved_prompts_df["response_" + prompt_name] = ""
        saved_prompts_df["isMatch_" + prompt_name] = ""


    
    # save response prompt and isMatch to df at index
    saved_prompts_df.at[index, "prompt_" + prompt_name] = prompt
    saved_prompts_df.at[index, "response_" + prompt_name] = response
    saved_prompts_df.at[index, "isMatch_" + prompt_name] = isMatch
    
    

In [11]:
# print(df_products)
df_products
print(str(df_products["longdescr"][0]))


the pocket size koss 3 band equalizer delivers high fidelity performance and output normally reserved for more expensive home systems with a 10db boost or 10db cut range of level it features a 3 band equalizer that allows for convenient and individual bass midrange and treble adjustment power output is greater than 20mw per channel providing clean and undistorted output into your favorite stereophones ergonomically designed for easy handling a rotary volume control and on off switch are placed for convenient usage 


In [12]:
def evaluate(df, prompt, temp=0, start = 0):
    # start = len(df) - start - 1
    if (MODEL == "gpt-3.5-turbo-instruct"):
        llm = OpenAI(model_name=MODEL, temperature=temp,max_tokens=MAX_TOKENS)
        print("Using OpenAI")
    else:
        llm = ChatOpenAI(model_name=MODEL, temperature=temp,max_tokens=MAX_TOKENS)
    for i in range(start, len(df)):
        print(i)
        # Creating Prompt and sending request and printing informations

        brandl = ""
        brandr = ""
        # if (df_products["brand"][df["id1"][i]-1]):
        #     brandl = "\nBrand: " + str(df_products["brand"][df["id1"][i]-1])
        # if (df_products["brand"][df["id2"][i]-1]):
        #     brandr = "\nBrand: " + str(df_products["brand"][df["id2"][i]-1])
        
        product1 = "Title: " + cut_length(str(df_products["title"][df["id1"][i]-1] or "none")) +brandl+"\nDescription: " + cut_length(str(df_products["longdescr"][df["id1"][i]-1] or "none")) 
        product2 = "Title: " + cut_length(str(df_products["title"][df["id2"][i]-1] or "none")) +brandr+ "\nDescription: " + cut_length(str(df_products["longdescr"][df["id2"][i]-1] or "none")) 

        chain = LLMChain(llm=llm, prompt=prompt, callbacks=[handler])
        
        try:
            response = chain.run(p1=product1, p2=product2, callbacks=[handler])        
        except Exception as e:
            print(e)
            i -= 1
            continue

        formatted_prompt = chain.prompt.format_prompt(p1=product1, p2=product2).text
        # if(i < 5):
        print(formatted_prompt)
        print("__\n__")
        print(response)
        print("__\n__")
        
        #Processing Response    
        isMatch = False
        if ("True" in response or "true" in response or "yes" in response):
            isMatch = True
        elif ("False" in response or "false" in response or "no" in response):
            isMatch = False
        
        df[prompt_name][i] = isMatch


        saveResponse(formatted_prompt, response, isMatch, i)

In [13]:
evaluate(df_mapping, prompt,temp=0, start=0)


0
Are the following two products the same?
Product 1: Title: optoma bl fp230c replacement projector lamp bulb with housing high quality replacement lamp
Description: this shopforbattery part number sfp 213 123271 is the premium projector lamp that replaced the optoma bl fp230c this projector lamp is a brand new lamp with new housing and tested to be 100 oem compatible it is different from other sellers that only 
Product 2: Title: optoma bl fp230c replacement projector lamp for optoma ep749 projector
Description: none

Answer with 'True' or 'False'.
__
__
False
__
__
1


/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_90230/2705043239.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[prompt_name][i] = isMatch


Are the following two products the same?
Product 1: Title: electrified bl fp230c sp 85r01gc01 replacement lamp with housing for optoma projectors
Description: brand new replacement projection lamp with brand new housing for optoma projectors 150 day electrified warranty
Product 2: Title: optoma bl fp230c replacement projector lamp for optoma ep749 projector
Description: none

Answer with 'True' or 'False'.
__
__
False
__
__
2


/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_90230/2705043239.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[prompt_name][i] = isMatch


Are the following two products the same?
Product 1: Title: optoma bl fp230c replacement lamp for ep749 ep719h
Description: projector lamps drive brightness of a projector over time as the lamp starts to fade the image of the projector will fade with it optoma lamps have been specifically designed for optoma s digital projectors to extend the life and lower the total cost
Product 2: Title: optoma bl fp230c replacement projector lamp for optoma ep749 projector
Description: none

Answer with 'True' or 'False'.
__
__
True
__
__
3


...

/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_90230/2705043239.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[prompt_name][i] = isMatch


Are the following two products the same?
Product 1: Title: urc r6 learning remote control
Description: rc r6 universal remote affordable control of your home theater with your new home theater you re ready to enjoy the best hollywood has to offer but you don t enjoy having a pile of remotes on your coffee table fortunately universal remote control has
Product 2: Title: level mount motorized full motion mount for flat screen tv s 10 42 screens
Description: lvm1009 features black aluminum and steel for lcd or plasma tvs 10 42 up to 100lbs built in bubble level stud finder all hardware included pan 30 degrees tilt 15 degrees and extend 10 5 cord management system neatly gathers cords vesa 75 100 200 400 

Answer with 'True' or 'False'.
__
__
False
__
__
78


/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_90230/2705043239.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[prompt_name][i] = isMatch


Are the following two products the same?
Product 1: Title: lens cleaning brush pen for camera lens viewfinder
Description: siig lens cleaning brush pen ac cn0512 s1 cleaning kits
Product 2: Title: level mount motorized full motion mount for flat screen tv s 10 42 screens
Description: lvm1009 features black aluminum and steel for lcd or plasma tvs 10 42 up to 100lbs built in bubble level stud finder all hardware included pan 30 degrees tilt 15 degrees and extend 10 5 cord management system neatly gathers cords vesa 75 100 200 400 

Answer with 'True' or 'False'.
__
__
False
__
__
79


/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_90230/2705043239.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[prompt_name][i] = isMatch


In [ ]:
df_mapping


,Unnamed: 0,id1,id2,label,prompt1_en_short_temp_0_gpt3.5-0301,match_prompt_manual_fewshot_1_eval2_temp_0_gpt3.5_0301,match_prompt_auto_fewshot_1_eval2_temp_0_gpt3.5_0301,match_prompt_auto_fewshot_2_ama_wal_eval2_temp_0_gpt3.5_0301,match_prompt_auto_fewshot_ama_wal_4examples_eval2_temp_0_gpt3.5_0301
0,0,7784,23390,True,True,False,False,False,False
1,1,15123,23390,True,False,False,False,False,True
2,2,21113,23390,True,True,True,True,False,True
3,3,7384,22973,True,True,True,True,True,True
4,4,11344,22973,True,False,True,False,True,True
...,...,...,...,...,...,...,...,...,...
400,400,23912,9467,False,False,False,False,False,False
401,401,23912,9468,False,False,False,False,False,False
402,402,23912,9469,False,False,False,False,False,False
403,403,23912,9470,False,False,False,False,False,False


In [ ]:
# saved_prompts_df

In [ ]:
# write df to csv
df_mapping.to_csv(PRODUCT_MAPPING, index=False)
saved_prompts_df.to_csv(SAVED_PROMPTS_PATH, index=False)
